In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import math
import skimage.io as imutils
%matplotlib inline
from keras.models import load_model
from keras.preprocessing import image as keras_preprocess
from keras.preprocessing import image as img
import os

In [ ]:
#Image structure = > (Height, Width, Color_Channel)
cat_image = Image.open('custom_test/dog.6.jpg')
#Load the Classifier
classifier = load_model('my_model.h5')

# Returns the labels for the classes according to the folder structre of clases
def get_labels_for_clases():
    #return ['car', 'cat', 'dog', 'shoe']
    return ['car' ,'cat', 'dog']

# Run prediction for a single image
def predict_for_single_image(image):
    #lable the images according the folder structure

    lables = get_labels_for_clases()
    out = classifier.predict_classes(image, verbose=0)
    return lables[out[0]]

#Prepreprocess image    
def preprocess_image(cropped_image_array):
    cropped_image = Image.fromarray(cropped_image_array)
    cropped_image = cropped_image.resize((64,64))
    cropped_image = keras_preprocess.img_to_array(cropped_image)
    cropped_image = np.expand_dims(cropped_image, axis=0)
    return cropped_image

In [ ]:
#save the image cordinates in a dictionary
image_cordinates_grid = {}

#if M x N window were to slide thought the image, identify the parameters for the loop condition
slide_window_height_and_width = (256, 256)

#Convert the image in to an numpy array
image_array = np.array(cat_image)

row_count_after_substraction = math.ceil(image_array.shape[0] / slide_window_height_and_width[0])
col_count_after_substraction = math.ceil(image_array.shape[1] / slide_window_height_and_width[1])

#Initialize x, y, w, h values
x, y, w, h = 0, 0, slide_window_height_and_width[1], slide_window_height_and_width[0]
count = 0
print("Image will be transformed in to a", row_count_after_substraction, "x", col_count_after_substraction, "grid")
for row in range(0, row_count_after_substraction):
    for col in range(0, col_count_after_substraction):
        
        #Change the X and Y locations respectivly to add the sliding window effect.
        cropped_image = image_array[y : y + h, x : x + w]
        saving_image = cropped_image
        image_cordinates_grid[str(count) + '_rect'] = (x, y, w, h)
        cropped_image = preprocess_image(cropped_image)
        image_cordinates_grid[str(count) + '_prediction'] = predict_for_single_image(cropped_image)
        x = x + w
        count = count + 1
    x = 0
    y = y + h

In [ ]:
image_cordinates_grid

In [ ]:
#Extract the relevent image cell to its corresponding prediction
iteration_size = math.ceil(len(image_cordinates_grid) / 2)
combined_grid = {}
#Compare adjesent cells for the same prediction and combine them
for i in range(0, iteration_size - 1):
    if image_cordinates_grid[str(i) + '_prediction'] == image_cordinates_grid[str(i + 1) + '_prediction']:
        #Join the regions
        cordinate_1 = image_cordinates_grid[str(i) + '_rect']
        cordinate_2 = image_cordinates_grid[str(i + 1) + '_rect']
        print("cordinate_1 : ",cordinate_1)
        print("cordinate_2 : ",cordinate_2)
        #Join in X axis
        
        if cordinate_1[1] == cordinate_2[1]:
           
            combined_grid[str(i) + '_rect'] = (cordinate_1[0], cordinate_1[1], cordinate_1[2] + cordinate_2[2], cordinate_1[3])
        #Join in Y axix
        
        if cordinate_1[0] == cordinate_2[0]:
            print("came to else if")
            combined_grid[str(i) + '_rect'] = (cordinate_1[0], cordinate_1[1],cordinate_1[2], cordinate_1[3] + cordinate_2[3])

In [ ]:
#Order the grid to have identical key numberings
combined_grid

In [ ]:
#Draw bounding boxes on the image from the identified regions

In [ ]:
combined_grid

In [ ]:
fig, ax =plt.subplots(1)

ax.imshow(image_array)

for key,value in combined_grid.items():
    x, y, w, h = value
    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='g', facecolor='none')
    ax.add_patch(rect)

plt.show()

In [ ]:
bounding_boxes = [len(combined_grid)]

for i, (key, value) in enumerate(combined_grid.items()):
    x, y, w, h = value
    prediction = preprocess_image(image_array[y : y + h, x : x + w])
    prediction = predict_for_single_image(prediction)
    bounding_boxes.append((prediction, np.array([x, y, w, h])))


In [ ]:
for predi, bbox in bounding_boxes[1:]:
    print(predi)

In [ ]:
#get all bounding boxes to an array
boundingbox_array = []

for i, (key, value) in enumerate(combined_grid.items()):
    x, y , w, h = value
    boundingbox_array.append(np.array((x,y,w,h)))
    
boundingbox_array = np.array(boundingbox_array)

In [ ]:
fig, ax =plt.subplots(1)
ax.imshow(image_array)

for (x,y,w,h) in pick:
    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='g', facecolor='none')
    plt.text(x, y, "dog")
    ax.add_patch(rect)

plt.show()

'b'

'#FFFFE0'